In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from torch import nn
from imblearn.over_sampling import SMOTE
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
# read csv
# https://research.unsw.edu.au/projects/unsw-nb15-dataset
# According to the website above csv is wrongly saved, so I change the name
# The number of records in the training set is 175,341 records and the testing set is 82,332 records from the different types, attack and normal.
df1 = pd.read_csv('../Data/UNSW-NB15/UNSW-NB15_1.csv', header=None)
df2 = pd.read_csv('../Data/UNSW-NB15/UNSW-NB15_2.csv', header=None)
df3 = pd.read_csv('../Data/UNSW-NB15/UNSW-NB15_3.csv', header=None)
df4 = pd.read_csv('../Data/UNSW-NB15/UNSW-NB15_4.csv', header=None)
#df_train, df_test = train_test_split(df, test_size=0.7)


C:\Users\hirot\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hirot\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,39,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(df1.shape + df2.shape + df3.shape + df4.shape)
df = pd.concat([df1, df2[1:700001], df3[1:700001], df4[1:440044]])

(700001, 49, 700001, 49, 700001, 49, 440044, 49)


In [4]:
df.shape

(2540044, 49)

In [5]:
features = pd.read_csv('../Data/UNSW-NB15/UNSW-NB15_features.csv', header=None)
df.columns = features[0]
df.head(10)

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0
5,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,NaN,0
6,59.166.0.6,2142,149.171.126.4,53,udp,CON,0.001134,132,164,31,...,0,12,7,1,2,2,1,1,NaN,0
7,10.40.182.3,0,10.40.182.3,0,arp,INT,0.000000,46,0,0,...,0,2,2,2,2,2,2,2,NaN,0
8,59.166.0.5,40726,149.171.126.6,53,udp,CON,0.001126,146,178,31,...,0,6,7,3,1,1,1,1,NaN,0
9,59.166.0.7,12660,149.171.126.4,53,udp,CON,0.001167,132,164,31,...,0,6,7,2,1,1,1,1,NaN,0


In [6]:
df.dtypes

0
srcip                object
sport                object
dstip                object
dsport               object
proto                object
state                object
dur                 float64
sbytes                int64
dbytes                int64
sttl                  int64
dttl                  int64
sloss                 int64
dloss                 int64
service              object
Sload               float64
Dload               float64
Spkts                 int64
Dpkts                 int64
swin                  int64
dwin                  int64
stcpb                 int64
dtcpb                 int64
smeansz               int64
dmeansz               int64
trans_depth           int64
res_bdy_len           int64
Sjit                float64
Djit                float64
Stime                 int64
Ltime                 int64
Sintpkt             float64
Dintpkt             float64
tcprtt              float64
synack              float64
ackdat              float64
is_sm_ips_ports   

In [7]:
df = df.fillna(0)
df.head(10)

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,0,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,0,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,0,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,0,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,0,0
5,59.166.0.0,32119,149.171.126.9,111,udp,CON,0.078339,568,312,31,...,0,2,4,2,3,1,1,2,0,0
6,59.166.0.6,2142,149.171.126.4,53,udp,CON,0.001134,132,164,31,...,0,12,7,1,2,2,1,1,0,0
7,10.40.182.3,0,10.40.182.3,0,arp,INT,0.000000,46,0,0,...,0,2,2,2,2,2,2,2,0,0
8,59.166.0.5,40726,149.171.126.6,53,udp,CON,0.001126,146,178,31,...,0,6,7,3,1,1,1,1,0,0
9,59.166.0.7,12660,149.171.126.4,53,udp,CON,0.001167,132,164,31,...,0,6,7,2,1,1,1,1,0,0


In [8]:
#ct_ftp_cmd must be integer
#ct_ftp_cmd           object
df["ct_ftp_cmd"]=df["ct_ftp_cmd"].fillna(0)
df["ct_ftp_cmd"]=df["ct_ftp_cmd"].str.replace("NaN", "0")
df["ct_ftp_cmd"]=df["ct_ftp_cmd"].str.replace(" " , "0")
df["ct_ftp_cmd"]=df["ct_ftp_cmd"].astype(float)

In [9]:
df["ct_ftp_cmd"]=df["ct_ftp_cmd"].fillna(0)
df.dtypes

0
srcip                object
sport                object
dstip                object
dsport               object
proto                object
state                object
dur                 float64
sbytes                int64
dbytes                int64
sttl                  int64
dttl                  int64
sloss                 int64
dloss                 int64
service              object
Sload               float64
Dload               float64
Spkts                 int64
Dpkts                 int64
swin                  int64
dwin                  int64
stcpb                 int64
dtcpb                 int64
smeansz               int64
dmeansz               int64
trans_depth           int64
res_bdy_len           int64
Sjit                float64
Djit                float64
Stime                 int64
Ltime                 int64
Sintpkt             float64
Dintpkt             float64
tcprtt              float64
synack              float64
ackdat              float64
is_sm_ips_ports   

In [10]:
print(df.isnull().sum())

0
srcip               0
sport               0
dstip               0
dsport              0
proto               0
state               0
dur                 0
sbytes              0
dbytes              0
sttl                0
dttl                0
sloss               0
dloss               0
service             0
Sload               0
Dload               0
Spkts               0
Dpkts               0
swin                0
dwin                0
stcpb               0
dtcpb               0
smeansz             0
dmeansz             0
trans_depth         0
res_bdy_len         0
Sjit                0
Djit                0
Stime               0
Ltime               0
Sintpkt             0
Dintpkt             0
tcprtt              0
synack              0
ackdat              0
is_sm_ips_ports     0
ct_state_ttl        0
ct_flw_http_mthd    0
is_ftp_login        0
ct_ftp_cmd          0
ct_srv_src          0
ct_srv_dst          0
ct_dst_ltm          0
ct_src_ ltm         0
ct_src_dport_ltm    0
ct_dst_s

In [11]:
#sport and dsport is numeric
pd.unique(df["sport"])
df["sport"]=df["sport"].str.replace("'" , "")
df["dsport"]=df["dsport"].str.replace("'" , "")

In [12]:
pd.unique(df["attack_cat"])
df["attack_cat"]=df["attack_cat"].str.replace("Backdoor" , "Backdoors")
df["attack_cat"]=df["attack_cat"].str.replace("Backdoorss" , "Backdoors")
#df["attack_cat"]=df["attack_cat"].str.replace("Backdoorsss" , "Backdoors")
df["attack_cat"]=df["attack_cat"].str.replace("Fuzzers " , "Fuzzers")
df["attack_cat"]=df["attack_cat"].str.replace("Shellcode " , "Shellcode")
df["attack_cat"]=df["attack_cat"].str.replace(" Shellcode" , "Shellcode")
df["attack_cat"]=df["attack_cat"].str.replace("Reconnaissance " , "Reconnaissance")
df["attack_cat"]=df["attack_cat"].str.replace(" Reconnaissance" , "Reconnaissance")
df["attack_cat"]=df["attack_cat"].str.replace('0', "Normal")
df["attack_cat"]=df["attack_cat"].fillna("Normal")
pd.unique(df["attack_cat"])

array(['Normal', 'Exploits', 'Reconnaissance', 'DoS', 'Generic',
       'Shellcode', ' Fuzzers', 'Worms', 'Backdoors', 'Analysis'],
      dtype=object)

In [13]:
# nominal to numeric of data
# srcip                object
# sport                object
# dstip                object
# dsport               object
# proto                 object
# service               object
# state                 object

# proto to numeric
# proto_mapping = {'xxx':2, 'xxx':1, 'xxx':0}
# data['proto'] = data['proto'].map(proto_mapping)

# one Hot Encoding is also applicable like using pandas dummies
#df_processed = pd.get_dummies(df, columns=(['proto','service','state']))

# srcip to numeric
srcip_le = LabelEncoder()
df['srcip'] = srcip_le.fit_transform(df['srcip'])
 
# sport to numeric
sport_le = LabelEncoder()
df['sport'] = sport_le.fit_transform(df['sport'])

# dstip to numeric
dstip_le = LabelEncoder()
df['dstip'] = dstip_le.fit_transform(df['dstip'])

# dsport to numeric
dsport_le = LabelEncoder()
df['dsport'] = dsport_le.fit_transform(df['dsport'])

# proto to numeric
proto_le = LabelEncoder()
df['proto'] = proto_le.fit_transform(df['proto'])
 
# service to numeric
service_le = LabelEncoder()
df['service'] = service_le.fit_transform(df['service'])

# state to numeric
state_le = LabelEncoder()
df['state'] = state_le.fit_transform(df['state'])

# nominal to numeric of data
# attack_cat            object

# target to numeric
attack_cat_le = LabelEncoder()
df['attack_cat'] = attack_cat_le.fit_transform(df['attack_cat'])
pd.set_option("display.max_columns", 200)

df.head(10)
# Adding columns number = 133 + 13 + 9 - 3  = 152


,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,Label
0,33,35745,24,47161,120,2,0.001055,132,164,31,29,0,0,2,500473.93750,621800.93750,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,3,7,1,3,1,1,1,6,0
1,33,35745,27,253,120,2,0.036133,528,304,31,29,0,0,0,87676.08594,50480.17188,4,4,0,0,0,0,132,76,0,0,9.891010,10.682733,1421927414,1421927414,7.005,7.564333,0.0,0.0,0.0,0,0,0.0,0.0,0.0,2,4,2,3,1,1,2,6,0
2,39,35745,25,47161,120,2,0.001119,146,178,31,29,0,0,2,521894.53130,636282.37500,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,12,8,1,2,2,1,1,6,0
3,38,35745,23,47161,120,2,0.001209,132,164,31,29,0,0,2,436724.56250,542597.18750,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.043,0.014000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,6,9,1,1,1,1,1,6,0
4,36,35745,8,47161,120,2,0.001169,146,178,31,29,0,0,2,499572.25000,609067.56250,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927414,1421927414,0.005,0.003000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,7,9,1,1,1,1,1,6,0
5,33,35745,27,1197,120,2,0.078339,568,312,31,29,0,0,0,43503.23438,23896.14258,4,4,0,0,0,0,142,78,0,0,29.682221,34.370340,1421927414,1421927414,21.003,24.315000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,2,4,2,3,1,1,2,6,0
6,39,35745,22,47161,120,2,0.001134,132,164,31,29,0,0,2,465608.46880,578483.25000,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927414,1421927414,0.017,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,12,7,1,2,2,1,1,6,0
7,2,35745,2,1,6,6,0.000000,46,0,0,0,0,0,0,0.00000,0.00000,1,0,0,0,0,0,46,0,0,0,0.000000,0.000000,1421927415,1421927415,0.000,0.000000,0.0,0.0,0.0,1,2,0.0,0.0,0.0,2,2,2,2,2,2,2,6,0
8,38,35745,24,47161,120,2,0.001126,146,178,31,29,0,0,2,518650.09380,632326.81250,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421927415,1421927415,0.018,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,6,7,3,1,1,1,1,6,0
9,40,35745,22,47161,120,2,0.001167,132,164,31,29,0,0,2,452442.15630,562125.06250,2,2,0,0,0,0,66,82,0,0,0.000000,0.000000,1421927415,1421927415,0.018,0.013000,0.0,0.0,0.0,0,0,0.0,0.0,0.0,6,7,2,1,1,1,1,6,0


In [14]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=0)

In [15]:
#df_train.dtypes
df_train.shape

(1778030, 49)

In [16]:
df_test.shape

(762014, 49)

In [17]:
#data_train = df_train.drop(['id', 'label', 'attack_cat'], axis=1)
data_train = df_train.drop(['Label', 'attack_cat'], axis=1)
label_train = df_train.iloc[:,-2]
data_train.head(10)

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
133504,37,35745,24,64374,114,5,0.004986,3944,2456,31,29,7,7,0,5.976735e+06,3.722423e+06,18,18,255,255,1340704394,1343802918,219,136,0,0,13.827065,0.365624,1421961841,1421961841,0.274471,0.260647,0.000722,0.000551,0.000171,0,0,0.0,0.0,0.0,7,6,3,7,1,1,3
568664,34,35745,27,64374,114,5,2.260782,13246,548216,31,29,21,197,0,4.667058e+04,1.935490e+06,230,438,255,255,2470017042,325566521,58,1252,0,0,2005.377637,1465.699179,1421949194,1421949197,9.870926,5.172355,0.000581,0.000458,0.000123,0,0,0.0,0.0,0.0,7,8,1,2,1,1,1
261748,33,35745,21,64374,114,5,1.144695,19306,1087890,31,29,2,370,5,1.345546e+05,7.592814e+06,362,746,255,255,3041141964,896802278,53,1458,1,524288,181.460154,112.254157,1421967324,1421967325,3.170003,1.535761,0.000670,0.000549,0.000121,0,0,1.0,0.0,0.0,2,1,3,1,1,1,1
392857,34,35745,24,58948,114,5,0.226398,8928,320,31,29,4,1,4,2.929708e+05,9.434712e+03,14,6,255,255,2337104779,2963379556,638,53,0,0,1597.246084,68.285023,1424260860,1424260860,17.415231,45.161602,0.000576,0.000460,0.000116,0,0,0.0,0.0,0.0,5,5,4,6,1,1,7
658729,34,35745,24,47161,120,2,0.001044,146,178,31,29,0,0,2,5.593869e+05,6.819923e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1424248339,1424248339,0.009000,0.007000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0.0,5,3,2,6,1,1,2
63739,37,35745,27,47161,120,2,0.001021,146,178,31,29,0,0,2,5.719882e+05,6.973556e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421929955,1421929955,0.003000,0.009000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0.0,2,2,4,2,1,1,1
318983,37,35745,24,64374,114,5,0.015679,2854,29226,31,29,7,17,0,1.424581e+06,1.460195e+07,46,48,255,255,2686777659,2687067983,62,609,0,0,19.584546,18.067135,1421969829,1421969829,0.340533,0.321872,0.000666,0.000540,0.000126,0,0,0.0,0.0,0.0,6,6,1,4,1,1,1
264293,29,35745,18,47161,120,6,0.000009,114,0,254,0,0,0,2,5.066666e+07,0.000000e+00,2,0,0,0,0,0,57,0,0,0,0.000000,0.000000,1424238176,1424238176,0.009000,0.000000,0.000000,0.000000,0.000000,0,2,0.0,0.0,0.0,32,32,32,32,32,13,33
149908,42,35745,26,64374,120,2,0.001019,146,178,31,29,0,0,0,5.731109e+05,6.987242e+05,2,2,0,0,0,0,73,89,0,0,0.000000,0.000000,1421932976,1421932976,0.001000,0.008000,0.000000,0.000000,0.000000,0,0,0.0,0.0,0.0,9,6,2,2,1,1,1
564522,42,35745,20,61234,114,5,5.577160,25992,1094788,31,29,38,390,0,3.720316e+04,1.568559e+06,464,858,255,255,2441176180,301438303,56,1276,0,0,4805.791745,3532.181831,1424226728,1424226733,12.043175,6.507236,0.000577,0.000456,0.000121,0,0,0.0,0.0,0.0,5,7,2,3,2,1,3


In [18]:
#min-max scaling
data_train_norm = (data_train - data_train.min()) / (data_train.max() - data_train.min())
data_train_norm = data_train_norm.fillna(0)

In [21]:
# x_data = torch.tensor(data.values, dtype=torch.float32)
# x_attack_cat = torch.tensor(attack_cat, dtype=torch.int32) 
train_X = torch.tensor(data_train_norm.values, dtype=torch.float32)
train_Y = torch.tensor(label_train.values, dtype=torch.long) 
train = TensorDataset(train_X, train_Y)

In [22]:
train_loader = DataLoader(train, batch_size=100, shuffle=True)
# drop_last = True

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(47, 850)
        self.fc2 = nn.Linear(850, 10)
        
    def forward(self, x):
        x = F.relu(self.fc1(x)) # ReLU: max(x, 0)
        x = self.fc2(x)
        # return F.log_softmax(x, dim=1)
        return x

model = Net()

In [24]:
print(torch.cuda.is_available())

True


In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(device)

cuda


In [26]:
criterion = nn.CrossEntropyLoss()

In [27]:
#optimizer = torch.optim.SGD(model.parameters(), lr=0.03)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03)

In [28]:
loss_list=[]
for epoch in range(100): #learning 100 times
    #total_loss = 0
    model.train()
    for train_x, train_y in train_loader:
        train_x, train_y = Variable(train_x), Variable(train_y)
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        optimizer.zero_grad()
        output = model(train_x)
        #output = model(train_x).softmax(dim=1)
        #print(output)
        #print(output.size())
        #print(train_y.size())
        #print(output)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        loss_list.append(loss.data)
        #print('epoch {}, loss {}'.format(epoch, loss.item()))
        #total_loss += loss.item() #loss.data[0]
    #if (epoch+1)%60 == 0:
        #print(epoch+1, total_loss)

In [29]:
with torch.no_grad():
    model.eval()
    correct = 0
    for test_x, test_y in train_loader:
        test_x, test_y = Variable(test_x), Variable(test_y)
        test_x = test_x.to(device)
        test_y = test_y.to(device)
        output = model(test_x)
        pred = torch.max(output.data, 1)[1]
        #print(pred)
        correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(train_loader.dataset)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 1719779/1778030 (97%)



In [31]:
testdata = df_test.drop(['Label', 'attack_cat'], axis=1)
testlabel = df_test.iloc[:,-2]

#min-max scaling
testdata_norm = (testdata - testdata.min()) / (testdata.max() - testdata.min())
testdata_norm = testdata_norm.fillna(0)

testdata_norm.head(10)

,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,service,Sload,Dload,Spkts,Dpkts,swin,dwin,stcpb,dtcpb,smeansz,dmeansz,trans_depth,res_bdy_len,Sjit,Djit,Stime,Ltime,Sintpkt,Dintpkt,tcprtt,synack,ackdat,is_sm_ips_ports,ct_state_ttl,ct_flw_http_mthd,is_ftp_login,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm
248337,0.214286,1.0,0.934783,1.000000,0.850746,0.133333,1.126369e-06,0.000292,0.000032,0.113725,0.122047,0.000208,0.000000,0.000000,3.837792e-04,0.006995,0.000416,0.000820,1.0,1.0,0.195004,0.692060,0.630319,0.034667,0.0,0.0,0.000003,2.146766e-06,0.004174,0.004159,5.018053e-05,2.028232e-05,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.015152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
170829,0.857143,1.0,0.565217,1.000000,0.850746,0.333333,1.845871e-06,0.000197,0.001584,0.121569,0.114173,0.001457,0.002553,0.000000,2.055203e-04,0.232259,0.004160,0.003645,1.0,1.0,0.514403,0.016366,0.042553,0.386667,0.0,0.0,0.000017,3.056192e-05,0.015375,0.015360,6.769806e-06,6.764442e-06,0.000083,0.000115,0.000056,0.0,0.000000,0.0,0.0,0.0,0.106061,0.060606,0.015152,0.015152,0.000000,0.000000,0.000000
110394,0.952381,1.0,0.195652,0.724314,0.895522,0.133333,4.095992e-06,0.000042,0.000021,0.121569,0.114173,0.000000,0.000000,0.000000,1.514560e-05,0.001054,0.000416,0.000364,0.0,0.0,0.000000,0.000000,0.090426,0.050667,0.0,0.0,0.000007,1.381530e-05,0.996142,0.996141,1.108204e-04,1.283790e-04,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.045455,0.045455,0.045455,0.000000,0.000000,0.000000,0.000000
502211,0.738095,1.0,0.326087,0.732609,0.895522,0.400000,1.024283e-09,0.000009,0.000000,0.996078,0.000000,0.000000,0.000000,0.166667,8.461366e-03,0.000000,0.000208,0.000000,0.0,0.0,0.000000,0.000000,0.037899,0.000000,0.0,0.0,0.000000,0.000000e+00,0.992661,0.992660,1.499750e-07,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.348485,0.348485,0.318182,0.318182,0.318182,0.355932,0.348485
262064,0.952381,1.0,0.586957,0.820984,0.850746,0.333333,3.225876e-05,0.000025,0.000129,0.121569,0.114173,0.000208,0.000365,0.333333,1.258488e-06,0.000969,0.000624,0.000729,1.0,1.0,0.291839,0.883826,0.035239,0.157333,0.0,0.0,0.002888,1.263554e-04,0.998004,0.998003,9.436062e-04,6.796035e-04,0.000055,0.000101,0.000018,0.0,0.000000,0.0,0.0,0.0,0.030303,0.030303,0.045455,0.045455,0.000000,0.000000,0.045455
171071,0.904762,1.0,0.543478,1.000000,0.850746,0.333333,2.089992e-06,0.000119,0.000112,0.121569,0.114173,0.000833,0.000729,0.000000,1.050529e-04,0.014073,0.001664,0.001640,1.0,1.0,0.978324,0.979452,0.063830,0.060667,0.0,0.0,0.000053,2.745285e-06,0.015378,0.015363,2.004778e-05,1.769196e-05,0.000059,0.000104,0.000022,0.0,0.000000,0.0,0.0,0.0,0.196970,0.181818,0.166667,0.166667,0.075758,0.000000,0.075758
450706,0.976190,1.0,0.195652,1.000000,0.850746,0.333333,5.614207e-07,0.000113,0.000118,0.121569,0.114173,0.001041,0.000729,0.000000,3.802456e-04,0.054145,0.002288,0.001276,1.0,1.0,0.883456,0.883530,0.044548,0.082000,0.0,0.0,0.000010,4.897563e-07,0.007649,0.007633,3.914431e-06,4.846708e-06,0.000071,0.000127,0.000025,0.0,0.000000,0.0,0.0,0.0,0.136364,0.075758,0.106061,0.106061,0.015152,0.016949,0.060606
509394,0.666667,1.0,0.260870,0.732609,0.895522,0.400000,1.024283e-09,0.000009,0.000000,0.996078,0.000000,0.000000,0.000000,0.166667,8.461366e-03,0.000000,0.000208,0.000000,0.0,0.0,0.000000,0.000000,0.037899,0.000000,0.0,0.0,0.000000,0.000000e+00,0.984049,0.984048,1.499750e-07,0.000000e+00,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0,0.0,0.363636,0.363636,0.090909,0.090909,0.090909,0.101695,0.363636
430874,0.690476,1.0,0.391304,0.018594,0.850746,0.333333,7.905128e-05,0.000044,0.000024,0.996078,0.992126,0.000416,0.000182,0.000000,9.771028e-07,0.000074,0.001040,0.000729,1.0,1.0,0.324374,0.959520,0.037234,0.029333,0.0,0.0,0.003760,1.917815e-04,0.991749,0.991748,1.249660e-03,1.605152e-03,0.003337,0.005792,0.001322,0.0,0.166667,0.0,0.0,0.0,0.000000,0.000000,0.000

In [32]:
test_X = torch.tensor(testdata_norm.values, dtype=torch.float32)
test_Y = torch.tensor(testlabel.values, dtype=torch.long) 

test = TensorDataset(test_X, test_Y)
test_loader = DataLoader(test, batch_size=100)

In [33]:
test_X.shape

torch.Size([762014, 47])

In [34]:
pred_list = list()
test_y_list = list()
with torch.no_grad():
    model.eval()
    correct = 0
    for test_x, test_y in test_loader:
        test_y_list.extend(test_y.to('cpu').detach().numpy().tolist())
        test_x, test_y = Variable(test_x), Variable(test_y)
        test_x = test_x.to(device)
        test_y = test_y.to(device)
        output = model(test_x)
        pred = torch.max(output.data, 1)[1]
        pred_list.extend(pred.to('cpu').detach().numpy().tolist())
        correct += pred.eq(test_y.data.view_as(pred)).sum() 
    # correct_rate
    data_num = len(test_loader.dataset)
    print('correct _rate: {}/{} ({:.0f}%)\n'.format(correct, data_num, 100. * correct / data_num))

correct _rate: 732898/762014 (96%)



In [35]:
accuracy_score(test_y_list, pred_list)

0.9617907282543365

In [36]:
precision_score(test_y_list, pred_list, average='macro')

C:\Users\hirot\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.37828085035617326

In [37]:
recall_score(test_y_list, pred_list, average='macro')

0.4100804243784005

In [38]:
f1_score(test_y_list, pred_list, average='macro')

0.36525742336649086

In [39]:
cm = confusion_matrix(test_y_list, pred_list, labels=[0,1,2,3,4,5,6,7,8,9])
print(cm)

[[  6520      0      0      0    767      2      2      9      0      0]
 [   663      0      0      0    141      0      0      5      0      0]
 [   499      0      0      0    205      1      0     10      0      0]
 [  3233      0      0      1   1729      7      0     38      0      0]
 [  4452      0      0      7   8866     11      2     52      0      0]
 [   601      0      0      1   1059  63239      9      7      0      0]
 [  8493      0      0      0   4930      4 651854      6      0      0]
 [   801      0      0      0    864      6      0   2418      0      0]
 [   159      0      0      0    283      1      0     10      0      0]
 [     2      0      0      0     44      1      0      0      0      0]]
